In [ ]:
from os import path
import cognee
import dspy
from cognee.modules.cognify.dataset import HotPotQA

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

await cognee.prune.prune_system()

colbertv2_wiki17_abstracts = dspy.ColBERTv2(url = "http://20.102.90.50:2017/wiki17_abstracts")
dspy.configure(rm = colbertv2_wiki17_abstracts)

# dataset = HotPotQA(
#     train_seed = 1,
#     train_size = 10,
#     eval_seed = 2023,
#     dev_size = 0,
#     test_size = 0,
#     keep_details = True,
# )

# texts_to_add = []

# for train_case in dataset.train:
#     train_case_text = "\r\n".join(" ".join(context_sentences) for context_sentences in train_case.get("context")["sentences"])

#     texts_to_add.append(train_case_text)

dataset_name = "short_stories"
await cognee.add("data://" + data_directory_path, dataset_name)


In [1]:
from os import path
import cognee

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

print(cognee.datasets.list_datasets())

train_dataset = cognee.datasets.query_data("short_stories")
print(len(train_dataset))

/Users/borisarzentar/Projects/Topoteretes/cognee/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['short_stories']
1


In [2]:
from os import path
import logging
import cognee

logging.basicConfig(level = logging.INFO)

await cognee.prune.prune_system()

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

await cognee.cognify("short_stories")

  self._variable_reference_to_variable.clear()
Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 28588.25it/s]
0.04s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


CancelledError: 

In [ ]:
import networkx as nx
import pandas as pd
import graphistry
from cognee.config import Config
from cognee.utils import render_graph
from cognee.infrastructure.databases.graph.get_graph_client import get_graph_client, GraphDBType

config = Config()
config.load()

graphistry.register(
    api = 3,
    username = config.graphistry_username,
    password = config.graphistry_password
)

graph_client = await get_graph_client(GraphDBType.NETWORKX, "cognee_graph.pkl")
graph = graph_client.graph

await render_graph(graph)

# edges = nx.to_pandas_edgelist(graph)

# nodes_data = [{
#     "id": node_id,
#     "label": node["name"] if "name" in node else node_id,
# } for (node_id, node) in graph.nodes(data = True)]

# nodes = pd.DataFrame(nodes_data)

# plotter = graphistry.edges(edges, source = "source", destination = "target").nodes(nodes, "id")

# plotter.bind(edge_title = "relationship_name", edge_label = "relationship_name", point_title = "name", point_label = "name")
# url = plotter.plot(render = False, as_files = True)
# print(f"Graph is visualized at: {url}")


In [ ]:
from os import path
import cognee
from cognee.infrastructure.databases.graph.get_graph_client import get_graph_client, GraphDBType
from cognee.modules.search.vector.search_similarity import search_similarity

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

graph_client = await get_graph_client(GraphDBType.NETWORKX)
graph = graph_client.graph

results = await search_similarity("Who are French girls?", graph)

for result in results:
    print("French girls" in result)
    print(result)